In [56]:
import pandas as pd
import requests
import numpy as np
import yaml
import json
import time

## NBA Data Pull

In [166]:
r = requests.get('https://www.balldontlie.io/api/v1/players', params={'per_page': 100})
j = json.loads(r.content.decode())
pd.DataFrame(j['data'])

r_list = [requests.get('https://www.balldontlie.io/api/v1/players', params={'per_page': 100, 'page': n}) for n in range(0, j['meta']['total_pages'])]

df_players = pd.concat([pd.json_normalize(json.loads(m.content.decode())['data']) for m in r_list])
df_players = df_players.sort_values('id').drop_duplicates().set_index('id')

df_players.to_csv('player_list.csv')

In [49]:
r = requests.get('https://www.balldontlie.io/api/v1/teams', params={'per_page': 100})
j = json.loads(r.content.decode())
pd.DataFrame(j['data'])

r_list = [requests.get('https://www.balldontlie.io/api/v1/teams', params={'per_page': 100, 'page': n}) for n in range(0, j['meta']['total_pages'])]

df_teams = pd.concat([pd.json_normalize(json.loads(m.content.decode())['data']) for m in r_list])
df_teams = df_teams.sort_values('id').drop_duplicates().set_index('id')

df_teams.to_csv('team_list.csv')

In [68]:
r = requests.get('https://www.balldontlie.io/api/v1/games', params={'per_page': 100})
j = json.loads(r.content.decode())
pd.DataFrame(j['data'])

#rate limit hecks you up here if you don't pace urself

r_list = []

for i in range(0, j['meta']['total_pages']):
    time.sleep(1)
    r_list.append(
        requests.get('https://www.balldontlie.io/api/v1/games',
                     params={'per_page': 100, 'page': i}
                     )
    )

df_games = pd.concat([pd.json_normalize(json.loads(m.content.decode())['data']) for m in r_list])
df_games = df_games.sort_values('id').drop_duplicates().set_index('id')

df_games.to_csv('game_list.csv')

In [74]:
r = requests.get('https://www.balldontlie.io/api/v1/stats', params={'per_page': 100})
j = json.loads(r.content.decode())
pd.json_normalize(j['data'])
j['meta']
#rate limit hecks you up here if you don't pace urself

{'total_pages': 11824,
 'current_page': 1,
 'next_page': 2,
 'per_page': 100,
 'total_count': 1182335}

#### yes we are actually unironically scraping everything all the way back to 1979. Sorry @hi_im_danny

In [81]:
r_list = []

for i in range(1, j['meta']['total_pages']):
    time.sleep(1)
    r = requests.get('https://www.balldontlie.io/api/v1/stats',
                     params={'per_page': 100, 'page': i}
                     )
    r_list.append(r)
    with open('stats_json/'+ str(i) + ".json", "w") as file:
        file.write(r.content.decode())

df_stats = pd.concat([pd.json_normalize(json.loads(m.content.decode())['data']) for m in r_list])
df_stats = df_stats.sort_values('id').drop_duplicates().set_index('id')

df_stats.to_csv('stats_list.csv')

In [93]:
# if you had to break up your requests, this will read in the cached data

c_list=[]
for i in range(1, 7929):
    with open('stats_json/'+ str(i) + '.json', 'r') as file:
        r = file.read()
        c_list.append(r)

df_stats = pd.concat([df_stats, pd.concat([pd.json_normalize(json.loads(m)['data']) for m in c_list])])

KeyboardInterrupt: 

In [126]:
df_stats = df_stats.convert_dtypes()
df_stats = df_stats.dropna(subset=['player.first_name'])
df_stats = df_stats.dropna(axis=1, how='all')
df_stats = df_stats.dropna()

In [167]:
df_stats.to_csv('stats_list.csv')